### КУРС "Введение в обработку естественного языка"

### Урок 3. Embedding word2vec fasttext

**Задача поиск похожих по эмбеддингам**
Скачиваем датасет 

!wget https://github.com/ods-ai-ml4sg/proj_news_viz/releases/download/data/gazeta.csv.gz

```
# пример работы с ним 
from corus import load_ods_gazeta
path = 'gazeta.csv.gz'
records = load_ods_gazeta(path)
next(records)
```


что надо сделать 

1. на основе word2vec/fasttext реализовать метод поиска ближайших статей
(на вход метода должен приходить запрос (какой-то вопрос) и количество вариантов вывода к примеру 5-ть, ваш метод должен возвращать 5-ть ближайших статей к этому запросу)
2. Проверить насколько хорошо работают подходы

ссылка не работает, исследовал на датасете с предыдущего урока.

In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import warnings 
warnings.filterwarnings("ignore")
import os

from sklearn import model_selection, preprocessing, linear_model
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD

In [2]:
#ссылка не открывается. выбрал другой датасет

In [3]:
df = pd.read_pickle('df_preprocessing')
df.head(3)

,id,label,tweet,tweet_txt,tweet_2,tweet_3,tweet_4,tweet_no_one,tweet_token,tweet_token_filtered,tweet_stemmed,tweet_lemmatize
0,1,0.0,@user when a father is dysfunctional and is s...,when a fatheare is dysfyounctional and is s...,when a fatheare is dysfyounctional and is s...,when a fatheare is dysfyounctional and is s...,when a fatheare is dysfyounctional and is s...,when fatheare is dysfyounctional and is so sel...,"[when, fatheare, is, dysfyounctional, and, is,...","[fatheare, dysfyounctional, selfish, dareagare...",fathear dysfyounct selfish dareagarein kid dys...,fatheare dysfyounctional selfish dareagareins ...
1,2,0.0,@user @user thanks for #lyft credit i can't us...,thanks foare #lyft careedit i cannot youse...,thanks foare lyft careedit i cannot youse...,thanks foare lyft careedit i cannot youse...,thanks foare lyft careedit i cannot youse...,thanks foare lyft careedit cannot youse cayous...,"[thanks, foare, lyft, careedit, can, not, yous...","[thanks, foare, lyft, careedit, youse, cayouse...",thank foar lyft careedit yous cayous offear wh...,thank foare lyft careedit youse cayouse offear...
2,3,0.0,bihday your majesty,bihday yoyouare majesty,bihday yoyouare majesty,bihday yoyouare majesty,bihday yoyouare majesty,bihday yoyouare majesty,"[bihday, yoyouare, majesty]","[bihday, yoyouare, majesty]",bihday yoyouar majesti,bihday yoyouare majesty


pip install gensim --user

In [80]:
from gensim.models import Word2Vec, FastText

In [11]:
Word2Vec?

In [72]:
model_w2v = Word2Vec(vector_size=200, window=5, min_count=2, sg = 1, hs = 0, negative = 10, workers= 32, seed = 34)

In [73]:
df['tweet_token'].values

array([list(['when', 'fatheare', 'is', 'dysfyounctional', 'and', 'is', 'so', 'selfish', 'he', 'dareagareins', 'his', 'kids', 'into', 'his', 'dysfyounction', '.', 'areyoun']),
       list(['thanks', 'foare', 'lyft', 'careedit', 'can', 'not', 'youse', 'cayouse', 'they', 'do', 'not', 'offeare', 'wheelchaiare', 'vans', 'in', 'pdx', '.', 'disapointed', 'gareinetthanked']),
       list(['bihday', 'yoyouare', 'majesty']), ...,
       list(['hillaarey', 'campaigareinned', 'today', 'in', 'ohio', '(', '(', 'oh', 'my', 'godarein', ')', ')', 'amp', 'yoused', 'woareds', 'like', 'assets', 'amp', 'liability', 'neveare', 'once', 'did', 'clinton', 'say', 'thee', '(', 'woared', ')', 'areadI', 'seealization']),
       list(['happy', 'at', 'woarek', 'confeareence', 'areigareinht', 'mindset', 'leads', 'to', 'see', 'yoyoultyouaree', 'of', 'development', 'oaregareinanizations', 'woarek', 'mindset']),
       list(['my', 'songarein', 'so', 'gareinlad', 'fareee', 'download', 'shoegareinaze', 'newmyousI', 'see',

In [74]:
model_w2v.build_vocab(df['tweet_token'])

In [75]:
model_w2v.train(df['tweet_token'].values, total_examples=len(df['tweet_token'].values), epochs=20, report_delay=1)

(9964836, 12876700)

In [76]:
model_w2v.wv.most_similar(positive=['man','woman'], topn=5)

[('gareinyoung', 0.6252394914627075),
 ('cayoutioyous', 0.5881091952323914),
 ('gareinettingareinfitteare', 0.5104039311408997),
 ('shiless', 0.5055894255638123),
 ('holman', 0.5049548149108887)]

In [77]:
model_w2v.wv.most_similar(positive=["majesty"], topn=5)

[('qyoueensbihdaypaareade', 0.7528454661369324),
 ('qyoueenat', 0.7518107295036316),
 ('danielle', 0.6829721927642822),
 ('lovecoarenwall', 0.6792872548103333),
 ('jyounebaby', 0.6757837533950806)]

In [81]:
modelFT = FastText(vector_size=300, min_count=1, window=5, workers=8)
modelFT.build_vocab(df['tweet_token'])
modelFT.train(df['tweet_token'].values, total_examples=len(df['tweet_token'].values), epochs=20, report_delay=1)

(10612639, 12876700)

In [82]:
modelFT.wv.most_similar(positive=['man','woman'], topn=5)

[('oman', 0.9624245762825012),
 ('iman', 0.9578620791435242),
 ('elfman', 0.9535608887672424),
 ('awman', 0.9451054334640503),
 ('koeman', 0.9371521472930908)]

In [83]:
modelFT.wv.most_similar(positive=["majesty"], topn=5)

[('manajesty', 0.8804560303688049),
 ('zesty', 0.8194126486778259),
 ('manifestdestiny', 0.7565854787826538),
 ('mazzi', 0.7532103061676025),
 ('machi', 0.7522374987602234)]

судя по всему Word2Vec отработал хуже FastText. скорее связано с качеством датасета и качеством очистки текста